In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# for creating captions file in the required format

csvfile = open("/content/drive/My Drive/Dataset - NEW/Door/1_doors.csv", 'r')
finfile = open("/content/drive/My Drive/Dataset - NEW/door_captions.txt", 'w')

caps = csvfile.read()
for line in caps.split('\n'):
    content = line.split(',')
    for i in range(1,4):
        if content[i] == '':
            print(content[0])
        finfile.write(content[0]+"#"+str(i)+" "+content[i]+'\n')    
finfile.close()
csvfile.close()

In [ ]:
# for merging different caption files into one.

allOurCaps = open("/content/drive/My Drive/Dataset - NEW/our_caps.txt", 'a')

finfile = open("/content/drive/My Drive/Dataset - NEW/capstoneIC_flickr/Captions.txt", 'r')
lines = finfile.read()

for i in lines:
  allOurCaps.write(i)

allOurCaps.close()
finfile.close()


In [ ]:
# to create test, dev and train datasets

import os
import random
files = os.listdir('/content/drive/My Drive/Dataset - NEW/capstoneIC_dataset')

flikr = []
ours = []
for i in files:
  if "image" == i[:5]:
    flikr.append(i)
  else:
    ours.append(i)

print(len(ours))
print(len(flikr))

testfile = open("/content/drive/My Drive/Dataset - NEW/capstoneIC_test.txt", 'w')
devfile = open("/content/drive/My Drive/Dataset - NEW/capstoneIC_dev.txt", 'w')
trainfile = open("/content/drive/My Drive/Dataset - NEW/capstoneIC_train.txt", 'w')

random.shuffle(ours)
random.shuffle(flikr)

test = ours[1800:2000]+flikr[2720:3020]
dev = ours[1600:1800]+flikr[2420:2720]
train = ours[:1600]+flikr[:2420]

print(len(test))
print(len(train))
print(len(dev))

random.shuffle(test)
random.shuffle(train)
random.shuffle(dev)

for i in test:
  testfile.write(i+'\n')
testfile.close()

for i in train:
  trainfile.write(i+'\n')
trainfile.close()

for i in dev:
  devfile.write(i+'\n')
devfile.close()

In [ ]:
import string

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# extract descriptions for images
def load_descriptions(doc):
    mapping = dict()
    # process lines
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        if len(line) < 2:
            continue
        # take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
        # remove filename from image id
        image_id = image_id.split('.')[0]
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping
 
def clean_descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove hanging 's' and 'a'
            desc = [word for word in desc if len(word)>1]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # store as string
            desc_list[i] =  ' '.join(desc)
 
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc
 
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
 
filename = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_captions.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, '/content/drive/MyDrive/Capstone Project/Dataset - NEW/descriptions.txt')

Loaded: 5022 
Vocabulary Size: 4691


In [ ]:
from os import listdir
from pickle import dump
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
import string
# extract features from each photo in the directory
def extract_features(directory):
    # load the model
    model = InceptionResNetV2()
    # re-structure the model
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # summarize
    print(model.summary())
    # extract features from each photo
    features = dict()
    for name in listdir(directory):
        # load an image from file
        filename = directory + '/' + name
        image = load_img(filename, target_size=(299, 299))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name.split('.')[0]
        # store feature
        features[image_id] = feature
        print('>%s' % name)
    return features
 
# extract features from all images
directory = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('/content/drive/MyDrive/Capstone Project/Dataset - NEW/features.pkl', 'wb'))

Streaming output truncated to the last 5000 lines.
>d92.jpg
>d94.jpg
>d93.jpg
>d95.jpg
>d97.jpg
>d96.jpg
>d98.png
>d99.jpg
>d100.jpg
>ch45.jpg
>ch42.jpg
>ch43.jpg
>ch44.jpg
>ch46.jpg
>ch47.jpg
>ch48.jpg
>ch49.jpg
>ch51.jpg
>ch52.png
>ch53.jpg
>ch50.png
>ch54.jpg
>ch55.jpg
>ch56.jpg
>ch57.jpg
>ch59.jpg
>ch60.jpg
>ch61.jpg
>ch62.jpg
>ch63.jpg
>ch58.jpg
>ch64.jpg
>ch65.jpg
>ch66.png
>im53.jpg
>im60.jpg
>im61.jpg
>im62.jpg
>ch67.jpg
>ch69.jpg
>ch70.jpg
>ch71.jpg
>ch72.jpg
>ch73.jpg
>ch74.jpg
>im64.jpg
>im65.jpg
>im66.jpg
>im67.jpg
>im68.jpg
>im69.jpg
>im70.jpg
>im71.jpg
>im72.jpg
>im73.jpg
>im74.jpg
>d101.jpg
>d102.jpeg
>im75.jpg
>im77.jpg
>im76.jpg
>im78.jpg
>im79.jpg
>im80.jpg
>im82.jpg
>im81.jpg
>d103.jpg
>d104.jpg
>d105.jpg
>d106.jpg
>d107.jpg
>d108.jpeg
>im89.jpg
>im88.jpg
>im87.jpg
>im86.jpg
>im84.jpg
>im83.jpg
>im85.jpg
>d109.JPG
>d110.JPG
>im91.JPG
>d111.JPG
>d112.JPG
>d113.JPG
>d114.JPG
>d115.JPG
>ch75.JPG
>d116.JPG
>ch76.JPG
>d117.JPG
>d118.JPG
>d119.JPG
>d120.JPG
>d121.JPG
>d122

In [ ]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
 
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
    # load all features
    all_features = load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[k] for k in dataset}
    return features
 
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# calculate the length of the description with the most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)
 
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos):
    X1, X2, y = list(), list(), list()
    # walk through each image identifier
    for key, desc_list in descriptions.items():
        # walk through each description for the image
        for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
            # split one sequence into multiple X,y pairs
            for i in range(1, len(seq)):
                # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # store
                X1.append(photos[key][0])
                X2.append(in_seq)
                y.append(out_seq)
    return array(X1), array(X2), array(y)
 
# define the captioning model
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(1536,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = GRU(256,return_sequences=True)(se2)
    se4 = LSTM(256,return_sequences=True)(se3)
    se5 = GRU(256,return_sequences=True)(se4)
    se6 = LSTM(256)(se5)
    # decoder model
    decoder1 = add([fe2, se6])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model
 
# train dataset
 
# load training dataset (6K)
filename = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_train.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/drive/MyDrive/Capstone Project/Dataset - NEW/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('/content/drive/MyDrive/Capstone Project/Dataset - NEW/features.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features)
 
# dev dataset
 
# load test set
filename = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_dev.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/content/drive/MyDrive/Capstone Project/Dataset - NEW/descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/content/drive/MyDrive/Capstone Project/Dataset - NEW/features.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare sequences
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features)
 
# fit model
 
# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=10, verbose=1, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

Dataset: 4020
Descriptions: train=4017
Photos: train=4020
Vocabulary Size: 4270
Description Length: 31
Dataset: 500
Descriptions: test=500
Photos: test=500
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 31, 256)      1093120     input_4[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 31, 256)      0           embedding[0][0]                  
__________________________________________________________________________________________________
gru (GRU)                       (No

In [ ]:
from pickle import dump
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
import nltk
from rouge_metric import PyRouge

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features
 
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	score = list()
 
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d for d in desc_list]
		actual.append(desc_list)
		predicted.append(yhat)
		#print(desc_list)
		#print(yhat)
		#s = nltk.translate.meteor_score.meteor_score(desc_list,yhat)
		#score.append(s)
  
  
  
  
	# calculate BLEU score
	# print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	# print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	# print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	# print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
	# print(actual)
	# print(predicted)
	# print(sum(score)/len(descriptions.items()))

	rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True, rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
	scores = rouge.evaluate(predicted, actual)
	print(scores)

 
 

			
      
# load training dataset (6K)
filename = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_train.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/drive/MyDrive/Capstone Project/Dataset - NEW/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open('/content/drive/MyDrive/Capstone Project/Dataset - NEW/tokenizer.pkl', 'wb'))
# obtain vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
 
# prepare test set
 
# load test set
filename = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_test.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/content/drive/MyDrive/Capstone Project/Dataset - NEW/descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/content/drive/MyDrive/Capstone Project/Dataset - NEW/features.pkl', test)
print('Photos: test=%d' % len(test_features))
 
# load the model
filename = '/content/drive/MyDrive/Capstone Project/Dataset - NEW/model-ep003-loss3.347-val_loss3.554.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 4020
Descriptions: train=4017
Vocabulary Size: 4270
Description Length: 31
Dataset: 499
Descriptions: test=499
Photos: test=499
{'rouge-1': {'r': 0.37639010716261345, 'p': 0.41509476097226305, 'f': 0.39479607976122455}, 'rouge-2': {'r': 0.10046613534001278, 'p': 0.1151571053746885, 'f': 0.10731115343216119}, 'rouge-4': {'r': 0.013233592797398051, 'p': 0.017519165314756488, 'f': 0.015077769550312663}, 'rouge-l': {'r': 0.3690773982408865, 'p': 0.40674670223592785, 'f': 0.38699755398687363}, 'rouge-w-1.2': {'r': 0.20312119858841562, 'p': 0.3520943620870565, 'f': 0.2576218460316146}, 'rouge-s4': {'r': 0.1034857627438175, 'p': 0.13129149660256645, 'f': 0.11574205019276011}, 'rouge-su4': {'r': 0.1487751542583003, 'p': 0.180547031007854, 'f': 0.16312847169626868}}


In [ ]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
from keras.models import load_model
 
# extract features from each photo in the directory
def extract_features(filename):
	# load the model
	model = InceptionResNetV2()

	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# load the photo
	image = load_img(filename, target_size=(299, 299))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature
 
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# load the tokenizer
tokenizer = load(open('/content/drive/MyDrive/Capstone Project/Dataset - NEW/tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 31
# load the model
model = load_model('/content/drive/MyDrive/Capstone Project/Dataset - NEW/model-ep003-loss3.335-val_loss3.561.h5')

# load and prepare the photograph
photo = extract_features('/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_dataset/ch318.jpg')
# generate description
description = generate_desc(model, tokenizer, photo, max_length)

print(description)

127.0.0.1 - - [30/Mar/2021 09:35:53] "POST /caption HTTP/1.1" 200 -


closed two door with two frame
startseq wooden chair with two cushion endseq


In [ ]:
def getCaptionObject(filename):
  # load the tokenizer
  tokenizer = load(open('/content/drive/MyDrive/Capstone Project/Dataset - NEW/tokenizer.pkl', 'rb'))
  # pre-define the max sequence length (from training)
  max_length = 31
  # load the model
  model = load_model('/content/drive/MyDrive/Capstone Project/Dataset - NEW/model-ep003-loss3.335-val_loss3.561.h5')
  # load and prepare the photograph
  photo = extract_features(filename)
  #photo = extract_features("/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_dataset/"+filename)
  # generate description
  description = generate_desc(model, tokenizer, photo, max_length)
  l = description.split(' ')
  description = ' '.join(l[1:len(l)-1])
  print(description)
  return description


In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, request, render_template,send_file
import os
from PIL import Image
import numpy
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app) 

@app.route('/', methods=['GET', 'POST'])
def test_request():
	return "connected..."


@app.route('/caption', methods=['GET', 'POST'])
def handle_request():
	file = request.files['file']
	print(file.filename)
	file.save(file.filename)
	ans = getCaptionObject(file.filename)
	return ans

@app.route('/ocr', methods=['GET', 'POST'])
def handle_request2():
	file = request.files['file']
	print(file.filename)
	file.save(file.filename)

	#audio = "cap.mp3"

	ans = getCaption(file.filename)
	print(ans)
	return ans


if __name__ == '__main__':
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e5cf593e5fd0.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Mar/2021 09:36:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2021 09:36:28] "GET /favicon.ico HTTP/1.1" 404 -


photo.jpg


...

wooden table with black chairs in front of it
d162.jpg


127.0.0.1 - - [30/Mar/2021 09:43:22] "POST /caption HTTP/1.1" 200 -


passage with many closed doors on either side of the passage
b76.jpg


127.0.0.1 - - [30/Mar/2021 09:44:30] "POST /caption HTTP/1.1" 200 -


double bed with pillows on it


In [ ]:
file1 = open('/content/drive/MyDrive/Capstone Project/Dataset - NEW/capstoneIC_test.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    ans = getCaptionObject(line)
    print("Line{}: {} - {}".format(count, line.strip(), ans))

man in black shirt and black shirt is standing on the street
Line1: image2961.jpg - man in black shirt and black shirt is standing on the street
man in red shirt is standing on the beach
Line2: image1000.jpg - man in red shirt is standing on the beach
man in blue jacket is standing on the water
Line3: image179.jpg - man in blue jacket is standing on the water
man standing on the staircase
Line4: im100.jpg - man standing on the staircase
man in black shirt is standing on the street
Line5: image1711.jpg - man in black shirt is standing on the street
football player in red jersey be be tackle player
Line6: image248.jpg - football player in red jersey be be tackle player
next children are standing on the beach
Line7: image2437.jpg - next children are standing on the beach
man in black shirt and black shirt and black shirt and two shirt and black shirt and two shirt and black shirt and black shirt and two shirt and black
Line8: image2346.jpg - man in black shirt and black shirt and black sh

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/Capstone Project/Dataset - NEW/model-ep003-loss3.335-val_loss3.561.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model) # Your model's name
tfl_model = converter.convert()
open( '/content/drive/MyDrive/Capstone Project/Dataset - NEW/model-ep003-loss3.335.tflite' , 'wb' ).write( tfl_model )


INFO:tensorflow:Assets written to: /tmp/tmp8w6il4i2/assets


INFO:tensorflow:Assets written to: /tmp/tmp8w6il4i2/assets


25366884

In [ ]:
!pip install -U nltk
import nltk
nltk.download('all')

Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.5)
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_

True

In [ ]:
import nltk
print(nltk.translate.meteor_score.meteor_score(["this is an apple", "that is an apple"], "an apple on this tree"))

0.6233062330623306


In [ ]:
pip uninstall nltk

Uninstalling nltk-3.5:
  Would remove:
    /usr/local/bin/nltk
    /usr/local/lib/python3.7/dist-packages/nltk-3.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/nltk/*
Proceed (y/n)? y
  Successfully uninstalled nltk-3.5


In [ ]:
pip install -U nltk


Processing /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306/nltk-3.5-cp37-none-any.whl


In [ ]:
pip install rouge-metric

     |████████████████████████████████| 153kB 5.8MB/s 


In [ ]:
from rouge_metric import PerlRouge

In [ ]:
from rouge_metric import PyRouge

# Load summary results
hypotheses = [
    'how are you\ni am fine',  # document 1: hypothesis
    'it is fine today\nwe won the football game',  # document 2: hypothesis
]
references = [[
    'how do you do\nfine thanks',  # document 1: reference 1
    'how old are you\ni am three',  # document 1: reference 2
], [
    'it is sunny today\nlet us go for a walk',  # document 2: reference 1
    'it is a terrible day\nwe lost the game',  # document 2: reference 2
]]

# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate(hypotheses, references)
print(scores)

{'rouge-1': {'r': 0.5182186234817814, 'p': 0.5555555555555556, 'f': 0.5362379555927943}, 'rouge-2': {'r': 0.19518716577540107, 'p': 0.2125, 'f': 0.20347597966879813}, 'rouge-4': {'r': 0.07142857142857142, 'p': 0.08333333333333333, 'f': 0.07692307692307691}, 'rouge-l': {'r': 0.5182186234817814, 'p': 0.5555555555555556, 'f': 0.5362379555927943}, 'rouge-w-1.2': {'r': 0.33608419409971513, 'p': 0.4734837712933738, 'f': 0.3931242798550236}, 'rouge-s4': {'r': 0.2549450549450549, 'p': 0.2916666666666667, 'f': 0.27207237393198186}, 'rouge-su4': {'r': 0.3149522799575822, 'p': 0.35526315789473684, 'f': 0.3338954468802698}}
